## What this notebook does (end‑to‑end):
	1.	Loads labels and a feature dataset (prefers mitsui_features_v2.parquet, falls back to mitsui_features.parquet, or rebuilds minimal label features if none found).
	2.	Merges labels safely (even if your feature file already contains a y column).
	3.	Trains one model per target (LightGBM if enough data, else ElasticNet) — all CPU‑friendly.
	4.	Implements the server‑style evaluation API predict(...):
	•	Uses precomputed features by (date_id, target) when available,
	•	Falls back to lag‑only features from the API when a date isn’t in your feature dataset.
	5.	Starts the Mitsui Inference Server for local gateway tests and for scoring.

## Why this structure?
	•	Stable & fast: minimal dependency footprint, CPU‑friendly.
	•	Leakage‑safe: all rolling features in the feature dataset were computed with .shift(1) (past‑only).
	•	Robust at submission: works even if your feature dataset isn’t attached — it will rebuild a basic label‑only set.

## Files expected
	•	Competition data (auto‑mounted):
	•	train_labels.csv (labels per date)
	•	Server files under kaggle_evaluation/* (the gateway & server scaffolding)
	•	Your feature dataset (recommended):
	•	mitsui_features_v2.parquet (preferred), or
	•	mitsui_features.parquet (fallback)
	•	If missing, the notebook rebuilds minimal label features into /kaggle/working/mitsui_features.parquet.
    Tip: keep a separate Feature Lab notebook to generate and version your feature parquet; attach it here via Add data.

## Pipeline at a glance

    Load labels → Load (or rebuild) features → Safe label merge → Train per‑target models → Start server → Predict batches
	•	LightGBM is used when a target has enough rows (fast, handles non‑linearities).
	•	ElasticNet is the fallback for smaller targets (linear, regularized).
	•	Prediction blend: 0.5 * carry_forward(y_lag_1) + 0.5 * model_pred for stability under Sharpe‑like metrics.

### 1) Imports, warnings, and small helpers
	•	Sets numpy/pandas warning filters to keep output clean.
	•	Utility functions:
	•	_find_one: glob helper to get the first matching file.
	•	_to_pd: converts Polars→Pandas when the gateway hands Polars frames.
	•	_ensure_one_row_targets: creates a single row with all target_* columns, replacing NaNs/±inf with 0 — avoids inference crashes.

2) Feature loading
	•	_load_features(labels_df) tries, in order:
	1.	/kaggle/working/mitsui_features_v2.parquet
	2.	/kaggle/working/mitsui_features.parquet
	3.	/kaggle/input/**/mitsui_features_v2.parquet
	4.	/kaggle/input/**/mitsui_features.parquet
	5.	Rebuilds a minimal feature set from train_labels.csv if none found.
	•	_rebuild_minimal_features: lags 1–4, rolling mean/std(5), sign of previous return. All past‑only and zero‑filled.

3) Safe supervised merge
	•	Melts labels to long form: (date_id, target, y).
	•	If your feature parquet already contains a y column, it’s dropped to avoid suffix conflicts.
	•	After merge, the code detects which y column to use (y, y_lbl, etc.) and standardizes to a single y.
	•	FEATURE_COLS are everything except keys and y.

4) Training (one model per target)
	•	If LightGBM is available and the target has ≥ ~200 rows → train LGBM with small leaves and 400 rounds.
	•	Else → ElasticNet with standardized inputs (StandardScaler).
	•	Trains quickly on CPU and prints a summary (how many used LGBM vs ElasticNet).

5) Predict‑time feature index
	•	Builds a lexsorted MultiIndex feats_idx on ["date_id","target"] for fast slicing during inference.
	•	This is crucial for the gateway loop to be efficient and avoid MultiIndex errors.

6) Online inference (predict(...))
	•	For each batch, we:
	•	Extract date_id from the batch “test” frame.
	•	Try precomputed features: feats_idx.xs(date_id, level="date_id", drop_level=False) → gives (target, features) for that date.
	•	If not available (e.g., future forecasting dates), we build minimal lag‑only features from the 4 lag tables that the API provides.
	•	Predict per target:
	•	Use the same feature names we trained on (usable_cols) — if the minimal set doesn’t have some columns, they’re just skipped.
	•	Blend model prediction with carry-forward (y_lag_1) for stability.
	•	Return a single-row wide DataFrame with target_0 ... target_423.

7) Start the Mitsui inference server
	•	Local gateway: runs a full dry‑run using the public data and prints progress (no internet needed).
	•	Scoring sandbox: Kaggle sets KAGGLE_IS_COMPETITION_RERUN=1; the code calls serve(), and the gateway handles submission.csv creation automatically.

In [1]:
# =========================================================
# MITSUI&CO. – E2E v2.1 (submit)
# - Robust feature loading (v2→v1, /working→/input; rebuild minimal if needed)
# - Merge labels safely (handles feature files that may already contain 'y')
# - Train per target (LightGBM if enough data else ElasticNet)
# - Server-style predict(): precomputed features by date_id,target → fallback to lag-only
# =========================================================

import os, gc, glob, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.seterr(invalid="ignore", divide="ignore", over="ignore")

# --------------------------
# Helpers
# --------------------------
def _find_one(patterns):
    for pat in patterns:
        hits = glob.glob(pat, recursive=True)
        if hits:
            return hits[0]
    return None

def _to_pd(df):
    try:
        import polars as pl
        if isinstance(df, pl.DataFrame):
            return df.to_pandas()
    except Exception:
        pass
    return df

def _ensure_one_row_targets(df, tcols):
    if df is None or len(df)==0:
        return pd.DataFrame({c:[0.0] for c in tcols})
    cols = [c for c in df.columns if str(c).startswith("target_")]
    if not cols:
        return pd.DataFrame({c:[0.0] for c in tcols})
    one = df[cols].head(1).copy().reindex(columns=tcols, fill_value=0.0)
    return one.replace([np.inf, -np.inf], np.nan).fillna(0.0)

def _rebuild_minimal_features(labels_df):
    long = labels_df.melt(id_vars="date_id", var_name="target", value_name="y")
    long["y"] = pd.to_numeric(long["y"], errors="coerce")
    parts = []
    for t, g in long.groupby("target", sort=False):
        df = g.sort_values("date_id").reset_index(drop=True)
        for L in range(1,5):
            df[f"y_lag_{L}"] = df["y"].shift(L)
        y_past = df["y"].shift(1)
        df["roll_mean_5"] = y_past.rolling(5).mean()
        df["roll_std_5"]  = y_past.rolling(5).std()
        df["sign_lag_1"]  = np.sign(y_past).fillna(0.0)
        feat_cols = [c for c in df.columns if c!="y"]
        df[feat_cols] = df[feat_cols].replace([np.inf,-np.inf], np.nan).fillna(0.0)
        df = df.dropna(subset=["y_lag_1","y_lag_2","y_lag_3","y_lag_4"]).reset_index(drop=True)
        parts.append(df)
        if len(parts)%100==0: gc.collect()
    feats = pd.concat(parts, axis=0, ignore_index=True)
    feats.to_parquet("/kaggle/working/mitsui_features.parquet", index=False)
    print("🔁 Rebuilt minimal features → /kaggle/working/mitsui_features.parquet")
    return feats

def _load_features(labels_df):
    # Prefer v2 then v1 in /working, then in /input
    p = _find_one([
        "/kaggle/working/mitsui_features_v2.parquet",
        "/kaggle/working/mitsui_features.parquet",
        "/kaggle/input/**/mitsui_features_v2.parquet",
        "/kaggle/input/**/mitsui_features.parquet",
    ])
    if p:
        print("Using features:", p)
        return pd.read_parquet(p)
    print("⚠️ No feature parquet found; rebuilding minimal label-only features...")
    return _rebuild_minimal_features(labels_df)

# --------------------------
# Load labels
# --------------------------
labels_path = _find_one(["/kaggle/input/**/train_labels.csv"])
assert labels_path is not None, "train_labels.csv not found. Attach the competition data."
labels = pd.read_csv(labels_path).sort_values("date_id").reset_index(drop=True)
target_cols = [c for c in labels.columns if c.startswith("target_")]
assert target_cols, "No target_* columns in train_labels.csv."

# --------------------------
# Load features (handles v2/v1 + working/input + rebuild)
# --------------------------
feats_long = _load_features(labels)

# --------------------------
# Supervised join (robust to 'y' already present in features)
# --------------------------
labels_long = labels.melt(id_vars="date_id", var_name="target", value_name="y")

# If features already have any y-like column, drop it to avoid suffix mess
feats_no_y = feats_long.drop(columns=[c for c in ["y","y_x","y_y","y_lbl"] if c in feats_long.columns], errors="ignore")

train_long = feats_no_y.merge(labels_long, on=["date_id","target"], how="left", suffixes=("", "_lbl"))

# Find the correct y column after merge
y_col = None
for cand in ["y","y_lbl","y_y","y_x"]:
    if cand in train_long.columns:
        y_col = cand; break
assert y_col is not None, "Could not find target column after merge."
train_long = train_long.rename(columns={y_col:"y"}).dropna(subset=["y"]).reset_index(drop=True)

FEATURE_COLS = [c for c in train_long.columns if c not in ["date_id","target","y"]]
assert FEATURE_COLS, "No feature columns found after merge."

# --------------------------
# Train per-target models
# --------------------------
HAS_LGB = True
try:
    import lightgbm as lgb
except Exception:
    HAS_LGB = False

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet

models, scalers, use_lgb = {}, {}, {}
for t, g in train_long.groupby("target", sort=False):
    X = g[FEATURE_COLS].to_numpy(dtype="float32")
    y = g["y"].to_numpy(dtype="float32")

    if HAS_LGB and len(g) >= 200:
        ds = lgb.Dataset(X, label=y)
        params = dict(
            objective="regression", metric="l2",
            learning_rate=0.05, num_leaves=16,
            min_data_in_leaf=25, feature_fraction=0.8,
            bagging_fraction=0.8, bagging_freq=1,
            verbosity=-1, num_threads=0
        )
        model = lgb.train(params, ds, num_boost_round=400)
        models[t] = model; use_lgb[t] = True
    else:
        scaler = StandardScaler()
        Xs = scaler.fit_transform(X)
        model = ElasticNet(alpha=0.0005, l1_ratio=0.1, random_state=42, max_iter=5000)
        model.fit(Xs, y)
        models[t] = model; scalers[t] = scaler; use_lgb[t] = False

    if len(models)%100==0: gc.collect()

print(f"✅ Trained {len(models)} targets; using LightGBM for {sum(use_lgb.values())}, ElasticNet for {len(models)-sum(use_lgb.values())}.")
print(f"Features used: {len(FEATURE_COLS)}")

# --------------------------
# Predict server (Mitsui API)
# --------------------------
# Build an index of features for quick lookup at predict time (drop 'y' if present)
# Build a lexsorted MultiIndex for fast/valid slicing
feats_no_y = feats_long.drop(columns=[c for c in ["y","y_x","y_y","y_lbl"] if c in feats_long.columns], errors="ignore")

# Ensure rows are ordered first, then set index and sort
feats_no_y = feats_no_y.sort_values(["date_id", "target"]).reset_index(drop=True)
feats_idx = feats_no_y.set_index(["date_id", "target"])[FEATURE_COLS].sort_index()

try:
    import polars as pl
except Exception:
    pl = None

def _minimal_lag_features(l1,l2,l3,l4, tcols):
    L1 = _ensure_one_row_targets(_to_pd(l1), tcols)
    L2 = _ensure_one_row_targets(_to_pd(l2), tcols)
    L3 = _ensure_one_row_targets(_to_pd(l3), tcols)
    L4 = _ensure_one_row_targets(_to_pd(l4), tcols)
    rows = []
    for t in tcols:
        lags = np.array([float(L1.at[0,t]), float(L2.at[0,t]), float(L3.at[0,t]), float(L4.at[0,t])], dtype="float32")
        rows.append({
            "target": t,
            "y_lag_1": lags[0],
            "y_lag_2": lags[1],
            "y_lag_3": lags[2],
            "y_lag_4": lags[3],
            "roll_mean_5": float(np.nanmean(lags)),
            "roll_std_5":  float(np.nanstd(lags)),
            "sign_lag_1":  float(np.sign(lags[0]) if np.isfinite(lags[0]) else 0.0),
        })
    return pd.DataFrame(rows).replace([np.inf,-np.inf], np.nan).fillna(0.0)

def _predict_for_date(date_id, lag1, lag2, lag3, lag4):
    # Use precomputed features if this date exists
    have_any = (date_id, target_cols[0]) in feats_idx.index
    if have_any:
        # xs() requires lexsorted index (we ensured above)
        sub = feats_idx.xs(date_id, level="date_id", drop_level=False).reset_index()
        # Now sub has columns: ['date_id', 'target', *feature_cols]
        feat_df = sub
    else:
        # Fallback: build minimal lag-only features on the fly
        feat_df = _minimal_lag_features(lag1, lag2, lag3, lag4, target_cols)
        feat_df.insert(0, "date_id", date_id)

    # Only use columns that were in training
    usable_cols = [c for c in FEATURE_COLS if c in feat_df.columns]
    preds = {}
    for _, r in feat_df.iterrows():
        t = r["target"]
        x = r[usable_cols].to_numpy(dtype="float32").reshape(1, -1)

        if t not in models:
            preds[t] = float(r.get("y_lag_1", 0.0))  # carry-forward fallback
            continue

        if use_lgb.get(t, False):
            yhat = float(models[t].predict(x)[0])
        else:
            xs = scalers[t].transform(x) if t in scalers else x
            yhat = float(models[t].predict(xs)[0])

        carry = float(r.get("y_lag_1", 0.0))
        preds[t] = 0.5 * carry + 0.5 * yhat  # stability blend

    return pd.DataFrame({t: [preds.get(t, 0.0)] for t in target_cols}).replace([np.inf, -np.inf], np.nan).fillna(0.0)

def predict(test, label_lags_1_batch, label_lags_2_batch, label_lags_3_batch, label_lags_4_batch):
    test_pd = _to_pd(test)
    date_id = int(test_pd["date_id"].iloc[0]) if (test_pd is not None and "date_id" in test_pd.columns and len(test_pd)>0) else -1
    out = _predict_for_date(date_id, label_lags_1_batch, label_lags_2_batch, label_lags_3_batch, label_lags_4_batch)
    return pl.DataFrame(out) if pl is not None else out

# --------------------------
# Start Mitsui server
# --------------------------
import kaggle_evaluation.mitsui_inference_server as mitsui_srv
inference_server = mitsui_srv.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("🔌 Starting Mitsui inference server (scoring)…")
    inference_server.serve()
else:
    # Find a data root that contains the local gateway assets
    def _find_comp_root():
        if os.path.exists("/kaggle/input/mitsui-commodity-prediction-challenge"):
            return "/kaggle/input/mitsui-commodity-prediction-challenge"
        # fallback: any folder with kaggle_evaluation inside
        for root, dirs, files in os.walk("/kaggle/input"):
            if "kaggle_evaluation" in dirs:
                return root
        return "/kaggle/input"
    comp_root = _find_comp_root()
    print("🧪 Local gateway run… using data at:", comp_root)
    inference_server.run_local_gateway((comp_root,))

Using features: /kaggle/input/feature-lab-mitsui-features-v2/mitsui_features_v2.parquet
✅ Trained 424 targets; using LightGBM for 424, ElasticNet for 0.
Features used: 24
🧪 Local gateway run… using data at: /kaggle/input/mitsui-commodity-prediction-challenge


## Next steps (optional improvements)
	•	Walk‑forward CV for LGBM/ElasticNet per target or per market group (LME/JPX/US/FX).
	•	Feature Lab upgrades: spreads, z‑spreads, EWMA vol, rolling corr/beta from train.csv + target_pairs.csv (already supported in mitsui_features_v2.parquet).
	•	Clipping/winsorization of extreme predictions; target‑family ensembling; decay‑weighted blends.